In [1]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv("EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
107430,"[2, 1453, 904, 431, 411, 265, 1191, 458, 370, ...","[2, 321, 3856, 294, 321, 1295, 302, 590, 1313,..."
14551,"[2, 333, 11054, 265, 6721, 224, 3, 0, 0, 0, 0,...","[2, 401, 8264, 323, 23762, 224, 3, 0, 0, 0, 0,..."
25273,"[2, 449, 304, 265, 10854, 224, 3, 0, 0, 0, 0, ...","[2, 1036, 323, 10816, 224, 3, 0, 0, 0, 0, 0, 0..."
101389,"[2, 300, 15215, 20263, 398, 1495, 334, 1148, 2...","[2, 321, 23763, 10837, 571, 1559, 318, 273, 21..."
63641,"[2, 1298, 12165, 1761, 300, 7120, 224, 3, 0, 0...","[2, 22344, 294, 5141, 346, 2027, 1919, 28997, ..."
...,...,...
81186,"[2, 290, 4045, 437, 369, 458, 265, 32139, 224,...","[2, 290, 372, 6404, 265, 369, 346, 410, 31431,..."
36782,"[2, 300, 2708, 334, 3338, 224, 3, 0, 0, 0, 0, ...","[2, 331, 2634, 318, 3918, 224, 3, 0, 0, 0, 0, ..."
19408,"[2, 14622, 3216, 582, 224, 3, 0, 0, 0, 0, 0, 0...","[2, 3664, 724, 7221, 519, 1392, 224, 3, 0, 0, ..."
111509,"[2, 2014, 2586, 290, 595, 803, 1666, 369, 332,...","[2, 1207, 3265, 290, 302, 969, 1614, 831, 265,..."


In [2]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
107430,"[2, 1453, 904, 431, 411, 265, 1191, 458, 370, ...","[2, 321, 3856, 294, 321, 1295, 302, 590, 1313,..."
14551,"[2, 333, 11054, 265, 6721, 224, 3, 0, 0, 0, 0,...","[2, 401, 8264, 323, 23762, 224, 3, 0, 0, 0, 0,..."
25273,"[2, 449, 304, 265, 10854, 224, 3, 0, 0, 0, 0, ...","[2, 1036, 323, 10816, 224, 3, 0, 0, 0, 0, 0, 0..."
101389,"[2, 300, 15215, 20263, 398, 1495, 334, 1148, 2...","[2, 321, 23763, 10837, 571, 1559, 318, 273, 21..."
63641,"[2, 1298, 12165, 1761, 300, 7120, 224, 3, 0, 0...","[2, 22344, 294, 5141, 346, 2027, 1919, 28997, ..."
...,...,...
81186,"[2, 290, 4045, 437, 369, 458, 265, 32139, 224,...","[2, 290, 372, 6404, 265, 369, 346, 410, 31431,..."
36782,"[2, 300, 2708, 334, 3338, 224, 3, 0, 0, 0, 0, ...","[2, 331, 2634, 318, 3918, 224, 3, 0, 0, 0, 0, ..."
19408,"[2, 14622, 3216, 582, 224, 3, 0, 0, 0, 0, 0, 0...","[2, 3664, 724, 7221, 519, 1392, 224, 3, 0, 0, ..."
111509,"[2, 2014, 2586, 290, 595, 803, 1666, 369, 332,...","[2, 1207, 3265, 290, 302, 969, 1614, 831, 265,..."


In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train_data = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test_data = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False, drop_last=True)

In [4]:
import json

with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [5]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=1, dropout=0.3):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

In [6]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size

        self.W1 = nn.Linear(hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, enc_out):
        hidden = hidden.unsqueeze(1)
        score = self.v(torch.tanh(self.W1(hidden) + self.W2(enc_out)))
        attention_weights = torch.softmax(score, dim=1)
        context_vector = torch.sum(attention_weights * enc_out, dim=1)
        return context_vector, attention_weights

In [60]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=1, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.lstm = nn.LSTM(
            embedding_dim + hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.attention = Attention(hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell, encoder_output):
        attention_vector, _ = self.attention(hidden[0], encoder_output)
        x = self.embedding(x)
        x = torch.cat((attention_vector, x), dim=-1)
        x = x.unsqueeze(1)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.fc(output)
        return output, hidden, cell

In [57]:
import torch.nn as nn
import torch
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, vocab_size, teacher_forcing_ratio=0.5):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.vocab_size = vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, src, trg):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(trg.shape[0], trg.shape[1], self.decoder.vocab_size).to("cuda")
        x = trg[:, 0]
        for t in range(1, trg.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, t, :] = output
            use_teacher_forcing = random.random() < self.teacher_forcing_ratio
            x = trg[:, t] if use_teacher_forcing else output.argmax(dim=1)

        return outputs
    
    def predict(self, src):
        enc_out, hidden, cell = self.encoder(src)
        outputs = torch.zeros(src.shape[0], src.shape[1], self.decoder.vocab_size).to("cuda")
        x = src[:, 0]
        for i in range(1, src.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell, enc_out)
            output = output.squeeze(1)
            outputs[:, i, :] = output
            x = output.argmax(dim=1)
        
        return outputs

In [15]:
def train(
        model,
        train_loader,
        test_loader,
        optimizer,
        criterion,
        device,
        scheduler, 
        num_epochs,
    ):
        prev_lr = optimizer.param_groups[0]["lr"]
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            model.encoder.train()
            model.decoder.train()
            epoch_loss = 0
            epoch_acc = 0
            progress_bar = tqdm(train_loader, desc="Training", leave=False)

            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)
                optimizer.zero_grad()
                outputs = model(src, trg)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, model.vocab_size), trg[:, 1:].reshape(-1)
                )
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

                preds = outputs.argmax(dim=2)
                correct = (preds == trg).float().sum().item()
                total = trg.numel()
                batch_acc = correct / total
                epoch_acc += batch_acc

                progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                )

            train_loss = epoch_loss / len(train_loader)
            train_acc = epoch_acc / len(train_loader)

            # ---------------- VALIDATION ---------------- #
            model.encoder.eval()
            model.decoder.eval()
            val_epoch_loss = 0
            val_epoch_acc = 0
            progress_bar = tqdm(test_loader, desc="Validating", leave=False)

            with torch.no_grad():
                for src, trg in progress_bar:
                    src, trg = src.to(device), trg.to(device)
                    outputs = model(src, trg)
                    loss = criterion(
                        outputs[:, 1:].reshape(-1, model.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                    val_epoch_loss += loss.item()

                    preds = outputs.argmax(dim=2)
                    correct = (preds == trg).float().sum().item()
                    total = trg.numel()
                    batch_acc = correct / total
                    val_epoch_acc += batch_acc

                    progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                    )

            val_loss = val_epoch_loss / len(test_loader)
            val_acc = val_epoch_acc / len(test_loader)
            print(
                f"Train loss : {train_loss}, Train accuracy : {train_acc}, Val_loss : {val_loss}, val accuracy : {val_acc}"
            )
            scheduler.step(val_loss)
            if scheduler.get_last_lr()[0] < prev_lr:
                prev_lr = scheduler.get_last_lr()[0]
                print(f"Encoder learning rate decreased to {prev_lr}")
        return train_loss, train_acc, val_loss, val_acc

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 32
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(seq2seq.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2
)

/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [21]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

train(
    seq2seq,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    30,
)


Epoch 1/30


Train loss : 5.960518540859833, Train accuracy : 0.0871590019917484, Val_loss : 5.336239510932855, val accuracy : 0.10713764441986229

Epoch 2/30


Train loss : 5.189315698821474, Train accuracy : 0.11342740788163329, Val_loss : 5.094479389950237, val accuracy : 0.11883526917404125

Epoch 3/30


Train loss : 4.965414310997488, Train accuracy : 0.12498944106558531, Val_loss : 4.9100702041018325, val accuracy : 0.1276790652654867

Epoch 4/30


Train loss : 4.778432626272439, Train accuracy : 0.13288141094038974, Val_loss : 4.7241049530231844, val accuracy : 0.13611956120943944

Epoch 5/30


...

Train loss : 2.846466505268014, Train accuracy : 0.23880752952055775, Val_loss : 3.180774716149389, val accuracy : 0.2228579000737464

Epoch 28/30


Train loss : 2.8118658081059573, Train accuracy : 0.24097044387537325, Val_loss : 3.1607614038264855, val accuracy : 0.22513558566863334

Epoch 29/30


Train loss : 2.7890387720090937, Train accuracy : 0.2421802754659268, Val_loss : 3.1294134137904748, val accuracy : 0.22732300884955758

Epoch 30/30


Train loss : 2.7472359966224347, Train accuracy : 0.2451406672357376, Val_loss : 3.1218040801782525, val accuracy : 0.2275361817846609


(2.7472359966224347,
 0.2451406672357376,
 3.1218040801782525,
 0.2275361817846609)

In [22]:
torch.save(seq2seq.state_dict(), "seq2seq30.pth")
torch.save(seq2seq.encoder.state_dict(), "encoder30.pth")
torch.save(seq2seq.decoder.state_dict(), "decoder30.pth")

In [23]:
train(
    seq2seq,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    30,
)
torch.save(seq2seq.state_dict(), "seq2seq60.pth")
torch.save(seq2seq.encoder.state_dict(), "encoder60.pth")
torch.save(seq2seq.decoder.state_dict(), "decoder60.pth")


Epoch 1/30


Train loss : 2.7337767390985035, Train accuracy : 0.2455380166097598, Val_loss : 3.1097962181125065, val accuracy : 0.22764564896755177

Epoch 2/30


Train loss : 2.7105355201701653, Train accuracy : 0.2469456892872388, Val_loss : 3.0924641157673523, val accuracy : 0.2291897123893805

Epoch 3/30


Train loss : 2.6726974009246436, Train accuracy : 0.24980327037985475, Val_loss : 3.070870278155909, val accuracy : 0.23093158493117014

Epoch 4/30


Train loss : 2.653861548317532, Train accuracy : 0.2509603072983357, Val_loss : 3.068830946905423, val accuracy : 0.23099688114552616

Epoch 5/30


...

Train loss : 2.3172351276065566, Train accuracy : 0.2716002454118651, Val_loss : 2.9082362820616865, val accuracy : 0.24120805678466084

Epoch 27/30


Train loss : 2.3158076916095083, Train accuracy : 0.2713812864561099, Val_loss : 2.9068793775761024, val accuracy : 0.24168049410029493

Epoch 28/30


Train loss : 2.295560368655128, Train accuracy : 0.27292122368046684, Val_loss : 2.9046109798735222, val accuracy : 0.24139242256637178

Epoch 29/30


Train loss : 2.283323816239605, Train accuracy : 0.27380483977094894, Val_loss : 2.905179496360036, val accuracy : 0.2412656710914454

Epoch 30/30


Train loss : 2.2857693877934495, Train accuracy : 0.2732768930502204, Val_loss : 2.9018685606728614, val accuracy : 0.24156334500983287


In [ ]:
a = Seq2Seq(encoder, decoder, vocab_size)
a.load_state_dict(torch.load("seq2seq.pth"))

<All keys matched successfully>

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 256
hidden_size = 32
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
seq2seq.load_state_dict(torch.load("seq2seq60.pth", weights_only=True))

/home/shusrith/projects/torch/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


<All keys matched successfully>

In [64]:
seq2seq.eval()
# with torch.no_grad():
a, b = next(iter(train_loader))
a = a.to(device)
b = b.to(device)
with torch.no_grad():
    x = seq2seq.predict(a)

x = x.argmax(dim=2)
x.shape

torch.Size([128, 18])

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer(
    "EnglishOrSpanish/vocab.json", "EnglishOrSpanish/merges.txt"
)

'<PAD> era un estudiante en tiempo <EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS><EOS>'

In [79]:
print(tokenizer.decode(b.tolist()[8]))
print(tokenizer.decode(x.tolist()[8]))

<SOS> tom debería ir a visitar a mary al hospital esta tarde <EOS><PAD><PAD><PAD><PAD>
<PAD> tom debería ir a ver a mary en el en esta tarde <EOS><EOS><EOS><EOS>
